In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io as sio
from sklearn import svm
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
import numpy as np

In [4]:
mat = sio.loadmat('./data/ex6data3.mat')
print(mat.keys())

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y', 'yval', 'Xval'])


In [5]:
trainning = pd.DataFrame(mat.get('X'), columns=['X1', 'X2'])
trainning['y'] = mat.get('y')
cv = pd.DataFrame(mat.get('Xval'), columns=['X1', 'X2'])
cv['y'] = mat.get('yval')

In [9]:
print(trainning.shape)
trainning.sample(3)

(211, 3)


,X1,X2,y
157,-0.207028,-0.535965,0
13,-0.246544,0.055556,1
139,-0.097120,0.213158,1


## Manual Grid Search for C and $\gamma$

In [10]:
candidate = [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100]

In [11]:
combination = [(C, gamma) for C in candidate for gamma in candidate]

In [12]:
search = []
for C, gamma in combination:
    svc = svm.SVC(C=C, gamma=gamma)
    svc.fit(trainning[['X1', 'X2']], trainning['y'])
    search.append(svc.score(cv[['X1', 'X2']], cv['y']))

In [15]:
best_score = search[np.argmax(search)]
best_param = combination[np.argmax(search)]
print(best_score, best_param)

0.965 (0.3, 100)


In [18]:
best_svc = svm.SVC(C=100, gamma=0.3)
best_svc.fit(trainning[['X1', 'X2']], trainning['y'])
y_pred = best_svc.predict(cv[['X1', 'X2']])
print(metrics.classification_report(cv['y'], y_pred))

             precision    recall  f1-score   support

          0       0.92      0.96      0.94       113
          1       0.94      0.89      0.91        87

avg / total       0.93      0.93      0.92       200



## Auto search for best hyper-parameter using GridSearchCV

In [20]:
paramters = {'C': candidate, 'gamma': candidate}
svc = svm.SVC()
clf = GridSearchCV(svc, paramters, n_jobs=-1)
clf.fit(trainning[['X1', 'X2']], trainning['y'])

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100], 'gamma': [0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [21]:
clf.best_params_

{'C': 10, 'gamma': 30}

In [22]:
clf.best_score_

0.9004739336492891